In [8]:
# ================================
# xDeepFM 复现项目 - Step 0 + Step 1
# 数据预处理 + 负采样 + 训练集构建
# ================================

import os
import random
import platform
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import Dataset, DataLoader

# -----------------------
# 基础配置
# -----------------------
data_path = "/workspace/FAR-Trans/FAR-Trans//"
FILES = {
    'customers': os.path.join(data_path, "customer_information.csv"),
    'assets': os.path.join(data_path, "asset_information.csv"),
    'transactions': os.path.join(data_path, "transactions.csv"),
    'markets': os.path.join(data_path, "markets.csv"),
    'limit_prices': os.path.join(data_path, "limit_prices.csv"),
    'close_prices': os.path.join(data_path, "close_prices.csv"),
}

SEED = 42
BATCH_SIZE = 512
NEG_SAMPLE_PER_POS = 4
DYNAMIC_NEG = False  # ❗已关闭：不再进行动态负采样（最初版本是用动态负采样，auc虚高，后转变为难负采样）

# ---- 设备信息（仅用于 DataLoader 缺省设定）
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IS_WINDOWS = (platform.system() == "Windows")

# ---- 随机性与稳定性
def set_seeds(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        import torch.backends.cudnn as cudnn
        cudnn.benchmark = True

set_seeds(SEED)

# -----------------------
# Step 0: 数据预处理
# -----------------------
print("加载数据...")
customers = pd.read_csv(FILES['customers'])
assets = pd.read_csv(FILES['assets'])
transactions = pd.read_csv(FILES['transactions'])
markets = pd.read_csv(FILES['markets'])
limit_prices = pd.read_csv(FILES['limit_prices'])
close_prices = pd.read_csv(FILES['close_prices'])
close_prices['timestamp'] = pd.to_datetime(close_prices['timestamp'])

# 仅保留 Buy 交易
transactions = transactions[transactions['transactionType'] == 'Buy']

# 用户与资产最新记录
customers_latest = customers.sort_values('timestamp').groupby('customerID').tail(1)
assets_latest = assets.sort_values('timestamp').groupby('ISIN').tail(1)

# 资产扩展特征
assets_latest = assets_latest.merge(
    markets[['marketID', 'country', 'marketClass', 'tradingHours']],
    on='marketID', how='left'
)
assets_latest[['country', 'marketClass', 'tradingHours']] = (
    assets_latest[['country', 'marketClass', 'tradingHours']].fillna('Unknown')
)

# limit_prices
assets_latest = assets_latest.merge(
    limit_prices[['ISIN', 'profitability', 'priceMinDate', 'priceMaxDate', 'minDate', 'maxDate']],
    on='ISIN', how='left'
)

# 数值处理
assets_latest['priceMinDate'] = pd.to_numeric(assets_latest['priceMinDate'], errors='coerce')
assets_latest['priceMaxDate'] = pd.to_numeric(assets_latest['priceMaxDate'], errors='coerce')
assets_latest['price_volatility_ratio'] = np.where(
    assets_latest['priceMinDate'] != 0,
    (assets_latest['priceMaxDate'] - assets_latest['priceMinDate']) / assets_latest['priceMinDate'],
    np.nan
)

assets_latest['minDate'] = pd.to_datetime(assets_latest['minDate'], errors='coerce')
assets_latest['maxDate'] = pd.to_datetime(assets_latest['maxDate'], errors='coerce')
assets_latest['listing_days'] = (assets_latest['maxDate'] - assets_latest['minDate']).dt.days

# close_prices 聚合
def extract_close_features(group):
    group = group.sort_values('timestamp')
    prices = group['closePrice'].values
    if len(prices) < 2:
        return pd.Series({'price_mean': np.nan, 'price_stability': np.nan,
                          'max_drawdown': np.nan, 'recent_return': np.nan})
    price_mean = prices.mean()
    price_std = prices.std()
    stability = price_std / price_mean if price_mean else np.nan
    cum_max = np.maximum.accumulate(prices)
    drawdowns = (prices / cum_max) - 1
    max_dd = drawdowns.min()
    recent_return = (prices[-1] - prices[max(-30, -len(prices))]) / prices[max(-30, -len(prices))]
    return pd.Series({'price_mean': price_mean, 'price_stability': stability,
                      'max_drawdown': max_dd, 'recent_return': recent_return})

close_feats = close_prices.groupby('ISIN').apply(extract_close_features).reset_index()
assets_latest = assets_latest.merge(close_feats, on='ISIN', how='left')

# 数值特征处理
num_cols = ['profitability', 'price_volatility_ratio', 'listing_days',
            'price_mean', 'price_stability', 'max_drawdown', 'recent_return']

imputer = SimpleImputer(strategy='median')
assets_latest[num_cols] = imputer.fit_transform(assets_latest[num_cols])

assets_latest['listing_days'] = np.log1p(assets_latest['listing_days'].clip(lower=0))
assets_latest['recent_return'] = assets_latest['recent_return'].clip(-1, 1)
assets_latest['max_drawdown'] = assets_latest['max_drawdown'].clip(-1, 0)
assets_latest['price_volatility_ratio'] = assets_latest['price_volatility_ratio'].clip(0, 5)

scaler = StandardScaler()
assets_latest[num_cols] = scaler.fit_transform(assets_latest[num_cols])

# 类别编码
def encode_cols(df, cols):
    encoders = {}
    for c in cols:
        df[c] = df[c].fillna('Unknown')
        le = LabelEncoder()
        df[c] = le.fit_transform(df[c].astype(str))
        encoders[c] = le
    return encoders

user_cat_features = ['customerType', 'riskLevel', 'investmentCapacity']
user_encoders = encode_cols(customers_latest, user_cat_features)

asset_cat_features = ['assetCategory', 'assetSubCategory', 'marketID',
                      'sector', 'industry', 'country', 'marketClass', 'tradingHours']
asset_encoders = encode_cols(assets_latest, asset_cat_features)

# 用户与资产索引
user2idx = {u: i for i, u in enumerate(customers_latest['customerID'].unique())}
asset2idx = {a: i for i, a in enumerate(assets_latest['ISIN'].unique())}
customers_latest['user_idx'] = customers_latest['customerID'].map(user2idx)
assets_latest['asset_idx'] = assets_latest['ISIN'].map(asset2idx)

print("清洗与特征标准化完成。")

# -----------------------
# Step 1: 样本构建 + 负采样 + 数据划分（难负采样：同市场优先 + 全局少量）
# -----------------------
# 映射 & 过滤
transactions = transactions[
    transactions['customerID'].isin(user2idx.keys()) &
    transactions['ISIN'].isin(asset2idx.keys())
].copy()
transactions['user_idx'] = transactions['customerID'].map(user2idx)
transactions['asset_idx'] = transactions['ISIN'].map(asset2idx)

missing_users = transactions['user_idx'].isna().sum()
missing_assets = transactions['asset_idx'].isna().sum()
print(f"可映射交易数: {len(transactions)}, 缺失user_idx: {missing_users}, 缺失asset_idx: {missing_assets}")

transactions = transactions.dropna(subset=['user_idx', 'asset_idx'])
transactions['user_idx'] = transactions['user_idx'].astype(int)
transactions['asset_idx'] = transactions['asset_idx'].astype(int)

print("构建正样本...")
pos_pairs = transactions[['user_idx', 'asset_idx']].drop_duplicates().copy()
pos_pairs['label'] = 1

num_users = len(user2idx)
num_items = len(asset2idx)
_rng = random.Random(SEED)  # 局部随机源

# ====== 硬负采样需要的市场索引（来自资产信息文件） ======
MARKET_RATIO = 0.9   # 同市场比例
GLOBAL_RATIO = 0.1   # 全局随机少量比例（补齐）
assert abs(MARKET_RATIO + GLOBAL_RATIO - 1.0) < 1e-6

_asset_meta = assets_latest[['asset_idx', 'marketID']].dropna().copy()
_asset_meta['asset_idx'] = _asset_meta['asset_idx'].astype(int)
_asset_meta['marketID']  = _asset_meta['marketID'].astype(int)

from collections import defaultdict
_assets_by_market = defaultdict(set)
for _, r in _asset_meta.iterrows():
    _assets_by_market[int(r['marketID'])].add(int(r['asset_idx']))

asset2market = dict(zip(_asset_meta['asset_idx'].tolist(), _asset_meta['marketID'].tolist()))
_all_items_set = set(range(num_items))  # 所有可用资产索引

def _split_counts(total_needed: int, market_ratio: float):
    mkt = int(round(total_needed * market_ratio))
    rnd = total_needed - mkt
    return mkt, rnd

# ====== 难负采样（同市场优先 + 全局补齐；严格负例） ======
def negative_sampling(pos_df, num_users, num_items, neg_per_pos=NEG_SAMPLE_PER_POS, rng_obj=_rng):
    pos_df = pos_df.astype({'user_idx': int, 'asset_idx': int})
    user_pos = pos_df.groupby('user_idx')['asset_idx'].apply(set).to_dict()
    neg_rows = []

    for u, pos_set in user_pos.items():
        if not pos_set:
            continue

        global_pool = list(_all_items_set - pos_set)
        if not global_pool:
            continue

        for a in pos_set:
            need_total = max(1, int(neg_per_pos))
            need_mkt, _ = _split_counts(need_total, MARKET_RATIO)

            mk_id = asset2market.get(int(a), None)
            mk_pool = list(((_assets_by_market.get(mk_id, set())) & _all_items_set) - pos_set) if mk_id is not None else []

            # 先采同市场
            take_mk = []
            if need_mkt > 0 and mk_pool:
                k = min(need_mkt, len(mk_pool))
                take_mk = rng_obj.sample(mk_pool, k)

            # 用全局补齐（剔除已交互 + 已选）
            already = set(take_mk) | pos_set
            rnd_pool = list(set(global_pool) - already)
            left = need_total - len(take_mk)
            take_rnd = []
            if left > 0 and rnd_pool:
                k = min(left, len(rnd_pool))
                take_rnd = rng_obj.sample(rnd_pool, k)

            sampled = take_mk + take_rnd

            # 若仍不足（极端小集合），再从全局补
            if len(sampled) < need_total:
                extra_pool = list(set(global_pool) - set(sampled))
                if extra_pool:
                    k = min(need_total - len(sampled), len(extra_pool))
                    sampled += rng_obj.sample(extra_pool, k)

            neg_rows.extend([(u, i) for i in sampled])

    neg_df = pd.DataFrame(neg_rows, columns=['user_idx', 'asset_idx'])
    neg_df['label'] = 0
    return neg_df

def build_train_dataframe(pos_df, num_users, num_items, neg_per_pos=NEG_SAMPLE_PER_POS):
    neg_df = negative_sampling(pos_df, num_users, num_items, neg_per_pos)
    data_df = pd.concat([pos_df, neg_df], axis=0, ignore_index=True)
    data_df = data_df.sample(frac=1.0, random_state=SEED).reset_index(drop=True)
    return data_df

data_df = build_train_dataframe(pos_pairs, num_users, num_items, NEG_SAMPLE_PER_POS)
print(f"正样本: {len(pos_pairs)}, 负样本: {len(data_df) - len(pos_pairs)}")

user_feat_df = customers_latest.set_index('user_idx')[user_cat_features].copy()
asset_feat_df = assets_latest.set_index('asset_idx')[asset_cat_features + num_cols].copy()

data_df = (
    data_df
    .merge(user_feat_df, left_on='user_idx', right_index=True, how='left')
    .merge(asset_feat_df, left_on='asset_idx', right_index=True, how='left')
)

assert data_df[user_cat_features + asset_cat_features + num_cols].isna().sum().sum() == 0, \
    "仍有缺失值，请检查预处理。"

train_df, valid_df = train_test_split(
    data_df,
    test_size=0.2,
    random_state=SEED,
    stratify=data_df['label']
)
print(f"Train 样本: {len(train_df)}, Valid 样本: {len(valid_df)}")

# -----------------------
# Dataset / DataLoader
# -----------------------
class FARTransDataset(Dataset):
    def __init__(self, df, user_cat_cols, asset_cat_cols, asset_num_cols):
        self.df = df.reset_index(drop=True)
        self.user_cat_cols = user_cat_cols
        self.asset_cat_cols = asset_cat_cols
        self.asset_num_cols = asset_num_cols

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        user_idx = torch.tensor(row['user_idx'], dtype=torch.long)
        asset_idx = torch.tensor(row['asset_idx'], dtype=torch.long)
        user_cat = torch.tensor(row[self.user_cat_cols].values.astype(np.int64))
        asset_cat = torch.tensor(row[self.asset_cat_cols].values.astype(np.int64))
        asset_num = torch.tensor(row[self.asset_num_cols].values.astype(np.float32))
        label = torch.tensor(row['label'], dtype=torch.float32)
        return user_idx, user_cat, asset_idx, asset_cat, asset_num, label

# ---- 通用 DataLoader 工厂（GPU 友好，CPU 保守）
def make_loader(dataset, batch_size, shuffle,
                num_workers: int = 0,
                pin_memory: bool = False,
                persistent_workers: bool = False,
                prefetch_factor: int | None = None):
    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        pin_memory=pin_memory,
        persistent_workers=persistent_workers if num_workers > 0 else False,
        prefetch_factor=(prefetch_factor if (prefetch_factor and num_workers > 0) else None),
    )

# 保持原行为（CPU 保守参数）
train_dataset = FARTransDataset(train_df, user_cat_features, asset_cat_features, num_cols)
valid_dataset = FARTransDataset(valid_df, user_cat_features, asset_cat_features, num_cols)
train_loader = make_loader(train_dataset, BATCH_SIZE, shuffle=True)
valid_loader = make_loader(valid_dataset, BATCH_SIZE, shuffle=False)

print(f"DataLoader 构建完成：train {len(train_loader)} 批次，valid {len(valid_loader)} 批次")

# 相较初版已删除：动态负采样挂钩（rebuild_train_loader_for_epoch）与 POS_PAIRS_TRAIN
print("Step 1 完成。可进入模型定义阶段。")

加载数据...


/tmp/ipykernel_974/1683348422.py:118: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  close_feats = close_prices.groupby('ISIN').apply(extract_close_features).reset_index()


清洗与特征标准化完成。
可映射交易数: 228913, 缺失user_idx: 0, 缺失asset_idx: 0
构建正样本...
正样本: 89884, 负样本: 359536
Train 样本: 359536, Valid 样本: 89884
DataLoader 构建完成：train 703 批次，valid 176 批次
Step 1 完成。可进入模型定义阶段。


In [9]:
## step2第二次调整
# ================================
# Step 2: 真实 2 层 CIN + xDeepFM 模型（稳定增强版）
# - CIN 每层 BatchNorm1d + GELU（防通道爆炸）
# - DNN: Linear + LayerNorm + GELU + Dropout（对稀疏更稳）
# - EmbeddingDropout 默认 0.05（可关）
# - 输出层 bias 可用正例基准率初始化（冷启动更稳）
# - logits 极性开关 logit_sign，AUC<0.5 时可一行翻转
# ================================

import math
import torch
import torch.nn as nn
import torch.nn.functional as F

# ===== 稳定默认超参（对齐友好 + 容量收敛） =====
EMBED_DIM  = 16          # 16 便于 bf16/fp16 对齐
CIN_SIZES  = [16, 16]    # 稳起步，稳定后可升到 [32, 32]
DNN_HIDDEN = [128, 64]
DROPOUT    = 0.35
EMB_DROPOUT_DEFAULT = 0.10  # 轻度随机化 emb，更稳

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---------- 轻量 EmbeddingDropout 钩子 ----------
class EmbeddingDropout(nn.Module):
    def __init__(self, p: float = 0.0):
        super().__init__()
        self.p = float(p)
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if not self.training or self.p <= 0.0:
            return x
        # x: (B, F, D)，逐元素 dropout
        return F.dropout(x, p=self.p, training=True)

class RealCIN(nn.Module):
    """
    真实的 CIN（Compressed Interaction Network）
    输入: x 形状 [B, F, D]
    过程: X_{k-1} 与 X_0 外积 -> 1x1 Conv 聚合 -> (B, H_k, D)
    输出: 拼接每层对 D 求和后的 (B, sum(H_k))
    """
    def __init__(self, field_num: int, embed_dim: int, layer_sizes):
        super().__init__()
        self.field_nums = [field_num]
        self.embed_dim = embed_dim
        self.convs = nn.ModuleList()
        self.bns   = nn.ModuleList()  # ✨ 每层做 BN 抑制通道尺度漂移
        for size in layer_sizes:
            in_channels = self.field_nums[-1] * self.field_nums[0]
            self.convs.append(nn.Conv1d(in_channels=in_channels,
                                        out_channels=size, kernel_size=1, bias=False))
            self.bns.append(nn.BatchNorm1d(size))
            self.field_nums.append(size)

    def forward(self, x):  # x: (B, F0, D)
        x0 = x
        xs = [x]
        outputs = []
        for conv, bn in zip(self.convs, self.bns):
            # 外积: (B, F_{k-1}, F0, D)
            z = torch.einsum('bfd,bhd->bfhd', xs[-1], x0)
            B, Fk_1, F0, D = z.size()
            z = z.reshape(B, Fk_1 * F0, D).contiguous()  # (B, C_in, L=D)
            z = conv(z)                                  # (B, H_k, D)
            z = bn(z)
            z = F.gelu(z)
            outputs.append(z.sum(dim=2))                 # (B, H_k)
            xs.append(z)                                 # (B, H_k, D)
        return torch.cat(outputs, dim=1) if outputs else torch.zeros(x.size(0), 0, device=x.device)

class XDeepFM(nn.Module):
    """
    xDeepFM（CIN + DNN）：
    - user_idx / asset_idx 各一个 Embedding（ID域）
    - 用户/资产类别特征：每列一个 Embedding
    - 数值特征：每列一个 Linear(1->D)，作为独立 field
    - CIN 显式交互 + DNN 隐式交互 -> 拼接输出 logits
    """
    def __init__(
        self,
        user_num: int,
        asset_num: int,
        embed_dim: int,
        user_feats_sizes: list,   # 每个用户类目的 vocab size
        asset_feats_sizes: list,  # 每个资产类目的 vocab size
        asset_num_feat_dim: int,  # 数值特征列数
        cin_sizes = CIN_SIZES,
        dnn_hidden = DNN_HIDDEN,
        dropout = DROPOUT,
        emb_dropout: float = EMB_DROPOUT_DEFAULT,
        base_pos_rate: float | None = None,  # 用正例基准率初始化输出 bias
        logit_sign: int = 1,                 # ✨ 极性开关：-1 可整体取反
    ):
        super().__init__()
        self.embed_dim  = embed_dim
        self.logit_sign = 1 if logit_sign >= 0 else -1

        # ID Embeddings
        self.user_emb  = nn.Embedding(user_num, embed_dim)
        self.asset_emb = nn.Embedding(asset_num, embed_dim)

        # Categorical Embeddings
        self.user_cat_embs  = nn.ModuleList([nn.Embedding(v, embed_dim) for v in user_feats_sizes])
        self.asset_cat_embs = nn.ModuleList([nn.Embedding(v, embed_dim) for v in asset_feats_sizes])

        # Numeric -> Embedding
        self.asset_num_projs = nn.ModuleList([nn.Linear(1, embed_dim) for _ in range(asset_num_feat_dim)])

        # field 总数
        self.field_num = 2 + len(user_feats_sizes) + len(asset_feats_sizes) + asset_num_feat_dim

        # CIN（显式交互）
        self.cin = RealCIN(field_num=self.field_num, embed_dim=embed_dim, layer_sizes=cin_sizes)
        cin_out_dim = sum(cin_sizes)

        # DNN（隐式交互）：Linear + LayerNorm + GELU + Dropout
        dnn_in_dim = self.field_num * embed_dim
        dnn_layers = []
        in_dim = dnn_in_dim
        for h in dnn_hidden:
            dnn_layers += [nn.Linear(in_dim, h), nn.LayerNorm(h), nn.GELU(), nn.Dropout(dropout)]
            in_dim = h
        self.dnn = nn.Sequential(*dnn_layers)

        # Embedding Dropout
        self.emb_dropout = EmbeddingDropout(p=emb_dropout)

        # 输出层
        self.output = nn.Linear(cin_out_dim + dnn_hidden[-1], 1)

        self._reset_params(base_pos_rate)

    def _reset_params(self, base_pos_rate: float | None):
        nn.init.xavier_uniform_(self.user_emb.weight)
        nn.init.xavier_uniform_(self.asset_emb.weight)
        for emb in self.user_cat_embs:  nn.init.xavier_uniform_(emb.weight)
        for emb in self.asset_cat_embs: nn.init.xavier_uniform_(emb.weight)
        for proj in self.asset_num_projs:
            nn.init.xavier_uniform_(proj.weight); nn.init.zeros_(proj.bias)
        for m in self.dnn:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight); nn.init.zeros_(m.bias)
        nn.init.xavier_uniform_(self.output.weight)
        if base_pos_rate is not None and 0.0 < base_pos_rate < 1.0:
            with torch.no_grad():
                bias = math.log(base_pos_rate / (1.0 - base_pos_rate))
                self.output.bias.fill_(bias)   # 冷启动更稳
        else:
            nn.init.zeros_(self.output.bias)

    def forward(self, user_idx, user_cat_feats, asset_idx, asset_cat_feats, asset_num_feats):

        B = user_idx.size(0)

        # ID 域
        user_e  = self.user_emb(user_idx).unsqueeze(1)    # (B, 1, D)
        asset_e = self.asset_emb(asset_idx).unsqueeze(1)  # (B, 1, D)

        # 类别域
        user_cat_list  = [emb(user_cat_feats[:, i]).unsqueeze(1)  for i, emb in enumerate(self.user_cat_embs)]
        asset_cat_list = [emb(asset_cat_feats[:, i]).unsqueeze(1) for i, emb in enumerate(self.asset_cat_embs)]

        # 数值域
        asset_num_feats = asset_num_feats.contiguous()
        asset_num_list = [proj(asset_num_feats[:, i:i+1]).unsqueeze(1) for i, proj in enumerate(self.asset_num_projs)]

        # 拼接 + EmbeddingDropout
        x = torch.cat([user_e, asset_e] + user_cat_list + asset_cat_list + asset_num_list, dim=1)  # (B, F, D)
        x = self.emb_dropout(x).contiguous()

        # CIN 显式交互
        cin_out = self.cin(x)  # (B, sum(CIN_SIZES))

        # DNN 隐式交互
        x_flat = x.reshape(B, -1).contiguous()
        dnn_out = self.dnn(x_flat) # (B, dnn_hidden[-1])

        # 拼接输出
        logits = self.output(torch.cat([cin_out, dnn_out], dim=1)).squeeze(1)  # (B,)
        return self.logit_sign * logits

# -------------------------------
# 构建模型实例
# -------------------------------

def _get_vocab_sizes(df: pd.DataFrame, cols: list) -> list:
    # vocab size = max(index) + 1；LabelEncoder 生成的是 0..K-1
    return [int(df[c].max()) + 1 for c in cols]

user_feats_sizes = _get_vocab_sizes(customers_latest, user_cat_features)
asset_feats_sizes = _get_vocab_sizes(assets_latest, asset_cat_features)

user_num = len(user2idx)
asset_num = len(asset2idx)
asset_num_feat_dim = len(num_cols)

model = XDeepFM(
    user_num=user_num,
    asset_num=asset_num,
    embed_dim=EMBED_DIM,
    user_feats_sizes=user_feats_sizes,
    asset_feats_sizes=asset_feats_sizes,
    asset_num_feat_dim=asset_num_feat_dim,
    cin_sizes=CIN_SIZES,
    dnn_hidden=DNN_HIDDEN,
    dropout=DROPOUT,
    emb_dropout=EMB_DROPOUT_DEFAULT,
    base_pos_rate=None,  # 若知道正例比例 p，填 p（如 0.12）
    logit_sign=1,        # 若“flipped AUC”更高，改为 -1
).to(DEVICE)

# ---------- 参数分组 helper：embedding vs others ----------
def get_param_groups_for_adamw(model: nn.Module):
    """
    返回两个 param 组，便于训练时：
      - 对 embedding 类参数设 weight_decay=0.0
      - 其它层用全局 weight_decay
    """
    emb_params, other_params = [], []
    for name, p in model.named_parameters():
        if not p.requires_grad:
            continue
        if ('emb' in name) or ('Embedding' in p.__class__.__name__):
            emb_params.append(p)
        else:
            other_params.append(p)
    return emb_params, other_params

def count_params(m):
    return sum(p.numel() for p in m.parameters() if p.requires_grad)

print("==== xDeepFM 模型构建完成 ====")
print(f"设备: {DEVICE}")
print(f"Embed 维度: {EMBED_DIM}")
print(f"Field 数量: 2(ID) + {len(user_cat_features)}(UserCat) + {len(asset_cat_features)}(AssetCat) + {len(num_cols)}(AssetNum) = {model.field_num}")
print(f"CIN 层: {CIN_SIZES} -> 输出维度 {sum(CIN_SIZES)}")
print(f"DNN 结构: {DNN_HIDDEN}")
print(f"可训练参数量: {count_params(model):,}")

==== xDeepFM 模型构建完成 ====
设备: cuda
Embed 维度: 16
Field 数量: 2(ID) + 3(UserCat) + 8(AssetCat) + 7(AssetNum) = 20
CIN 层: [16, 16] -> 输出维度 32
DNN 结构: [128, 64]
可训练参数量: 543,905


In [10]:
# ================================
# Step 3: 训练循环（可多进程加速）【稳健加速版】
# - AMP + TF32 + torch.compile
# - DataLoader 在 Notebook/Linux 优先使用 fork，少量 worker 更稳
# - Windows 自动降级单进程
# ================================
import os, gc, sys, math, platform, numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import roc_auc_score, log_loss

EPOCHS = 6
LR = 3e-4
WEIGHT_DECAY = 1e-5
GRAD_CLIP_NORM = 5.0
BEST_PATH = "xdeepfm_best.pt"

MONITOR = "auc"; MODE = "max"
EARLY_STOP = True
PATIENCE = 2
MIN_DELTA = 5e-4

USE_SCHED = True
LR_MIN = 1e-5
SCHED_CFG = dict(
    mode='max', factor=0.5, patience=1, cooldown=1,
    threshold=1e-4, threshold_mode='abs', min_lr=LR_MIN
)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
USE_AMP = (DEVICE.type == "cuda")
BF16_OK = (USE_AMP and torch.cuda.is_bf16_supported())
print(f"[Device] {DEVICE}; AMP={USE_AMP}")

# —— 数值加速：TF32 / compile ——
try:
    import torch.backends.cudnn as cudnn
    torch.backends.cuda.matmul.allow_tf32 = True
    cudnn.allow_tf32 = True
    torch.set_float32_matmul_precision('high')
except Exception:
    pass

if USE_AMP:
    try:
        # 在原配置上保持 reduce-overhead（兼容性更好）
        model = torch.compile(model, mode="reduce-overhead")
        print("✅ torch.compile enabled (reduce-overhead)")
    except Exception as e:
        print("⚠️ torch.compile failed:", e)

# —— 一键开关：多进程 DataLoader ——
MULTIWORKER = True          # 可按需关闭
FORCE_MULTIPROC = True      
def _in_notebook(): return ('ipykernel' in sys.modules) or ('IPython' in sys.modules)
IS_WINDOWS = (platform.system() == "Windows")

# 默认批大小：GPU 稍大、CPU 保守
BATCH_SIZE = 512 if USE_AMP else 128

if MULTIWORKER and (FORCE_MULTIPROC or (not _in_notebook())) and (not IS_WINDOWS):
    # ✅ 在 Linux/WSL 上优先使用 fork，能规避 notebook + spawn 的 pickling 问题
    try:
        import multiprocessing as mp
        mp.set_start_method("fork", force=True)
        start_method = "fork"
    except Exception as e:
        print("[DataLoader] set_start_method('fork') 失败，回退 spawn：", e)
        try:
            import multiprocessing as mp
            mp.set_start_method("spawn", force=True)
            start_method = "spawn"
        except Exception as e2:
            print("[DataLoader] set_start_method('spawn') 也失败，退回单进程：", e2)
            NUM_WORKERS = 0
            PIN_MEMORY  = USE_AMP
            PERSISTENT  = False
            PREFETCH    = None
            start_method = "none"
    # 仅当成功设置了启动方式才配置多进程参数
    if 'start_method' in locals() and start_method in ("fork", "spawn"):
        NUM_WORKERS = 8          # 不用太多，稳为主
        PIN_MEMORY  = USE_AMP
        PERSISTENT  = False      # Notebook 下建议先关
        PREFETCH    = 1
        print(f"[DataLoader] multiprocess ON: workers={NUM_WORKERS}, "
              f"persistent={PERSISTENT}, prefetch={PREFETCH}, start_method={start_method}")
else:
    # 单进程更稳（Windows / 显式关闭 / Notebook 未强制）
    NUM_WORKERS = 0
    PIN_MEMORY  = USE_AMP
    PERSISTENT  = False
    PREFETCH    = None
    if MULTIWORKER and _in_notebook() and not FORCE_MULTIPROC:
        print("[DataLoader] Notebook 环境：自动降级 workers=0（更稳）")
    elif MULTIWORKER and IS_WINDOWS:
        print("[DataLoader] Windows 环境：自动降级 workers=0")

def _build_loader(ds, shuffle):
    return DataLoader(
        ds, batch_size=BATCH_SIZE, shuffle=shuffle,
        num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY,
        persistent_workers=(PERSISTENT if NUM_WORKERS>0 else False),
        prefetch_factor=(PREFETCH if (PREFETCH and NUM_WORKERS>0) else None)
    )

# 直接复用 Step 1 产出的 train_dataset / valid_dataset
train_loader = _build_loader(train_dataset, True)
valid_loader = _build_loader(valid_dataset, False)

# —— 优化器/调度器/AMP ——
def _group_adamw(m, lr, wd, fused):
    emb_ids = set()
    for mod in m.modules():
        if isinstance(mod, nn.Embedding):
            for p in mod.parameters(recurse=False):
                emb_ids.add(id(p))
    emb, other = [], []
    for p in m.parameters():
        if not p.requires_grad: continue
        (emb if id(p) in emb_ids else other).append(p)
    try:
        opt = torch.optim.AdamW(
            [{"params": emb, "weight_decay": 0.0},
             {"params": other, "weight_decay": wd}],
            lr=lr, fused=fused)
    except TypeError:
        opt = torch.optim.AdamW(
            [{"params": emb, "weight_decay": 0.0},
             {"params": other, "weight_decay": wd}],
            lr=lr)
    return opt

optimizer = _group_adamw(model, LR, WEIGHT_DECAY, fused=USE_AMP)
scheduler = ReduceLROnPlateau(optimizer, **SCHED_CFG) if USE_SCHED else None
scaler = torch.amp.GradScaler('cuda', enabled=USE_AMP)

# —— 类别不平衡估计 ——
@torch.no_grad()
def estimate_pos_weight(loader, sample_batches: int = 10):
    pos = neg = checked = 0
    for batch in loader:
        y = batch[-1]
        y = torch.nan_to_num(y, nan=0.0, posinf=1.0, neginf=0.0)
        pos += int((y > 0.5).sum().item())
        neg += int((y <= 0.5).sum().item())
        checked += 1
        if checked >= sample_batches: break
    pos = max(pos, 1); neg = max(neg, 1)
    pw = neg / pos
    print(f"[ClassStats] pos={pos}, neg={neg}, pos_weight≈{pw:.2f}")
    return torch.tensor([pw], device=DEVICE, dtype=torch.float32)

POS_WEIGHT   = estimate_pos_weight(train_loader, sample_batches=10)
LABEL_SMOOTH = 0.05

def _to_device(batch):
    nb = (DEVICE.type == 'cuda')
    return tuple(t.to(DEVICE, non_blocking=nb) for t in batch)

def train_one_epoch():
    model.train()
    tot_loss = tot_samples = 0
    skipped = 0
    pbar = tqdm(train_loader, desc="Train", dynamic_ncols=True, mininterval=0.3, leave=False)
    for batch in pbar:
        batch = _to_device(batch)
        if batch[-1].numel() == 0:
            skipped += 1; continue
        optimizer.zero_grad(set_to_none=True)
        with torch.amp.autocast(device_type=DEVICE.type, enabled=USE_AMP,
                                dtype=torch.bfloat16 if BF16_OK else torch.float16):
            logits = torch.clamp(model(*batch[:-1]), -20, 20)
            y = torch.nan_to_num(batch[-1], nan=0.0, posinf=1.0, neginf=0.0)
            targets = y * (1.0 - LABEL_SMOOTH) + (1.0 - y) * LABEL_SMOOTH
            per_sample = F.binary_cross_entropy_with_logits(
                logits, targets, reduction='none', pos_weight=POS_WEIGHT)
        mask = torch.isfinite(per_sample)
        if not mask.any():
            skipped += 1; continue
        loss = per_sample[mask].mean()
        scaler.scale(loss).backward()
        if GRAD_CLIP_NORM and GRAD_CLIP_NORM > 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP_NORM)
        scaler.step(optimizer); scaler.update()
        tot_loss   += loss.item() * int(mask.sum().item())
        tot_samples += int(mask.sum().item())
        pbar.set_postfix(loss=f"{(tot_loss/max(1,tot_samples)):.5f}")
    pbar.close()
    if skipped: print(f"[WARN] train skipped {skipped} batches")
    return tot_loss / max(1, tot_samples)

@torch.no_grad()
def evaluate():
    model.eval()
    tot_loss = tot_samples = 0
    skipped = 0
    all_logits, all_labels = [], []
    pbar = tqdm(valid_loader, desc="Valid", dynamic_ncols=True, mininterval=0.5, leave=False)
    with torch.amp.autocast(device_type=DEVICE.type, enabled=USE_AMP,
                            dtype=torch.bfloat16 if BF16_OK else torch.float16):
        for batch in pbar:
            batch = _to_device(batch)
            if batch[-1].numel() == 0:
                skipped += 1; continue
            logits = torch.clamp(model(*batch[:-1]), -20, 20)
            y = torch.nan_to_num(batch[-1], nan=0.0, posinf=1.0, neginf=0.0)
            targets = y * (1.0 - LABEL_SMOOTH) + (1.0 - y) * LABEL_SMOOTH
            per_sample = F.binary_cross_entropy_with_logits(
                logits, targets, reduction='none', pos_weight=POS_WEIGHT)
            mask = torch.isfinite(per_sample)
            if not mask.any():
                skipped += 1; continue
            loss = per_sample[mask].mean()
            tot_loss += loss.item() * int(mask.sum().item())
            tot_samples += int(mask.sum().item())
            all_logits.append(logits[mask].detach().cpu().float().numpy())
            all_labels.append(y[mask].detach().cpu().numpy())
    pbar.close()
    if skipped: print(f"[WARN] valid skipped {skipped} batches")
    if tot_samples == 0: return math.nan, math.nan, math.nan
    logits = np.concatenate(all_logits); labels = np.concatenate(all_labels).astype(np.int64)
    probs = np.clip(1.0/(1.0+np.exp(-logits)), 1e-7, 1-1e-7)
    try: auc = roc_auc_score(labels, probs)
    except Exception: auc = math.nan
    try: ll = log_loss(labels, probs)
    except Exception: ll = math.nan
    return (tot_loss / tot_samples), auc, ll

def _is_better(curr, best):
    if best is None: return True
    delta = curr - best if MODE == "max" else best - curr
    return delta > MIN_DELTA

history = {"train_loss": [], "valid_loss": [], "valid_auc": [], "valid_logloss": []}
best_metric = None
print(f"开始训练：EPOCHS={EPOCHS}, BATCH_SIZE={BATCH_SIZE}, workers={NUM_WORKERS}, pin={PIN_MEMORY}, persistent={(PERSISTENT if NUM_WORKERS>0 else False)}")

for epoch in range(1, EPOCHS+1):
    print(f"\nEpoch {epoch}/{EPOCHS}")
    tr = train_one_epoch()
    vl, va, ll = evaluate()
    history["train_loss"].append(tr)
    history["valid_loss"].append(vl)
    history["valid_auc"].append(va)
    history["valid_logloss"].append(ll)
    mon = va if MONITOR=="auc" else (-ll)
    print(f"[Epoch {epoch:02d}] TrainLoss={tr:.5f} | ValidLoss={vl:.5f} | AUC={va:.5f} | LogLoss={ll:.5f}")

    if _is_better(mon, best_metric):
        best_metric = mon
        torch.save(model.state_dict(), BEST_PATH)
        print(f"🟢 New best ({MONITOR}): save -> {BEST_PATH}")

    if USE_SCHED and scheduler is not None:
        scheduler.step(va if not math.isnan(va) else -1.0)
        cur_lr = min(g['lr'] for g in optimizer.param_groups)
        if cur_lr <= LR_MIN and not _is_better(mon, best_metric):
            print(f"⏹ LR 已到下限({LR_MIN:.1e})且指标无提升，提前停止。")
            break

    if EARLY_STOP and epoch >= PATIENCE:
        recent = history["valid_auc"][-PATIENCE:] if MONITOR=="auc" else [-x for x in history["valid_logloss"][-PATIENCE:]]
        if len(recent)==PATIENCE and max(recent)-min(recent) < MIN_DELTA:
            print(f"⏹ Early stop triggered at epoch {epoch}")
            break

if best_metric is None and not os.path.exists(BEST_PATH):
    torch.save(model.state_dict(), BEST_PATH)
print(f"\n✅ 训练完成；最佳模型保存在 {BEST_PATH}")

[Device] cuda; AMP=True
✅ torch.compile enabled (reduce-overhead)
[DataLoader] multiprocess ON: workers=8, persistent=False, prefetch=1, start_method=fork
[ClassStats] pos=1044, neg=4076, pos_weight≈3.90
开始训练：EPOCHS=6, BATCH_SIZE=512, workers=8, pin=True, persistent=False

Epoch 1/6


[Epoch 01] TrainLoss=0.94924 | ValidLoss=0.86738 | AUC=0.88997 | LogLoss=0.51499
🟢 New best (auc): save -> xdeepfm_best.pt

Epoch 2/6


[Epoch 02] TrainLoss=0.86183 | ValidLoss=0.85455 | AUC=0.89877 | LogLoss=0.50571
🟢 New best (auc): save -> xdeepfm_best.pt

Epoch 3/6


[Epoch 03] TrainLoss=0.80915 | ValidLoss=0.85728 | AUC=0.89672 | LogLoss=0.48661

Epoch 4/6


[Epoch 04] TrainLoss=0.76291 | ValidLoss=0.87533 | AUC=0.88876 | LogLoss=0.46923

Epoch 5/6


[Epoch 05] TrainLoss=0.72335 | ValidLoss=0.89767 | AUC=0.88157 | LogLoss=0.49263

Epoch 6/6


[Epoch 06] TrainLoss=0.70953 | ValidLoss=0.90440 | AUC=0.87766 | LogLoss=0.48301

✅ 训练完成；最佳模型保存在 xdeepfm_best.pt


In [11]:
# ================================
# Step 3B: 验证集快速评估 + ROC/PR 绘图 + 分组分析（健壮版，含PRF）
# 依赖：model, DEVICE, USE_AMP, BEST_PATH, valid_dataset, train_df
# 产物：
#   - xdeepfm_ROC.png / xdeepfm_PR.png
#   - valid_y_true.npy / valid_y_score.npy
#   - xdeepfm_cls_report.csv（分类指标汇总）
#   - Warm/Cold 用户资产分析、热门度分组评估
#   - 单类分组健壮性回退
#   - Precision/Recall/F1（macro/micro）@ 固定阈值与最佳F1阈值
# ================================
import os, time, platform, warnings
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from sklearn.metrics import (
    roc_curve, auc, precision_recall_curve, average_precision_score,
    roc_auc_score, log_loss, precision_score, recall_score, f1_score
)
from sklearn.exceptions import UndefinedMetricWarning

warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

print("\n[Step3B] 开始评估与绘图（使用验证集）")

# ---- 加载 best 权重（若存在）----
if os.path.exists(BEST_PATH):
    state = torch.load(BEST_PATH, map_location="cpu")
    if isinstance(state, dict):
        model.load_state_dict(state)
        print(f"[Step3B] 已加载最佳权重：{BEST_PATH}")
    else:
        print(f"[Step3B] 读取 {BEST_PATH} 结果非 state_dict，跳过加载。")
else:
    print(f"[Step3B] 未找到 {BEST_PATH}，使用当前内存中的模型参数。")

model.to(DEVICE)
model.eval()

# ---- Eval DataLoader（验证阶段可用更大 batch）----
IS_WINDOWS = (platform.system() == "Windows")
CPU_COUNT = os.cpu_count() or 8
EVAL_BATCH_SIZE = 2048 if USE_AMP else 512
NUM_WORKERS = (max(2, CPU_COUNT // 2) if not IS_WINDOWS else 0) if USE_AMP else 0
PIN_MEMORY  = bool(USE_AMP)
PERSISTENT  = bool(USE_AMP and NUM_WORKERS > 0)
PREFETCH    = 2 if (USE_AMP and NUM_WORKERS > 0) else None

def make_eval_loader(dataset):
    kwargs = dict(
        dataset=dataset,
        batch_size=EVAL_BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        persistent_workers=PERSISTENT
    )
    if PREFETCH is not None:
        kwargs["prefetch_factor"] = PREFETCH
    return DataLoader(**kwargs)

valid_loader_eval = make_eval_loader(valid_dataset)

# ---- 小工具：更健壮的指标计算 ----
def _class_counts(y):
    y = np.asarray(y).astype(int)
    pos = int(np.sum(y == 1))
    neg = int(np.sum(y == 0))
    return pos, neg

def quick_metrics(y_true, y_score, ks_bins=100):
    """
    健壮指标：
    * 只有两类同时存在时才计算 AUC/AP/KS；
    * log_loss 显式 labels=[0,1]，若只有一类返回 np.nan；
    * Brier 总能算。
    """
    y_true = np.asarray(y_true).astype(int)
    y_score = np.asarray(y_score).astype(float)
    y_score = np.clip(y_score, 1e-7, 1-1e-7)

    out = {}
    pos, neg = _class_counts(y_true)
    both = (pos > 0 and neg > 0)

    if both:
        out["auc"] = float(roc_auc_score(y_true, y_score))
        out["ap"]  = float(average_precision_score(y_true, y_score))
    else:
        out["auc"] = np.nan
        out["ap"]  = np.nan

    try:
        out["logloss"] = float(log_loss(y_true, y_score, labels=[0, 1]))
        if not both:
            out["logloss"] = np.nan
    except Exception:
        out["logloss"] = np.nan

    if both:
        bins = np.linspace(0, 1, ks_bins+1)
        idx = np.digitize(y_score, bins) - 1
        tpr = np.zeros(ks_bins); fpr = np.zeros(ks_bins)
        for b in range(ks_bins):
            yb = y_true[idx == b]
            if yb.size > 0:
                tpr[b] = np.sum(yb == 1)
                fpr[b] = np.sum(yb == 0)
        tpr = np.cumsum(tpr) / max(pos, 1)
        fpr = np.cumsum(fpr) / max(neg, 1)
        out["ks"] = float(np.max(tpr - fpr))
    else:
        out["ks"] = np.nan

    out["brier"] = float(np.mean((y_score - y_true)**2))
    return out

def fmt(x):
    if x is None or (isinstance(x, float) and (np.isnan(x) or np.isinf(x))):
        return "N/A"
    return f"{x:.4f}"

def prf_with_thresholds(y_true, y_score, fixed_thr=0.5):
    """
    返回两组 PRF：固定阈值 & 最优F1阈值（在验证集上由 PR 曲线搜索）
    若数据只有单一类别，返回 N/A。
    """
    y_true = np.asarray(y_true).astype(int)
    y_score = np.asarray(y_score).astype(float)
    pos, neg = _class_counts(y_true)
    both = (pos > 0 and neg > 0)

    def _val(v):
        return np.nan if (v is None or (isinstance(v, float) and (np.isnan(v) or np.isinf(v)))) else float(v)

    out = {
        "fixed_thr": fixed_thr,
        "fixed": {"prec_macro": np.nan, "rec_macro": np.nan, "f1_macro": np.nan,
                  "prec_micro": np.nan, "rec_micro": np.nan, "f1_micro": np.nan},
        "bestF1_thr": np.nan,
        "bestF1": {"prec_macro": np.nan, "rec_macro": np.nan, "f1_macro": np.nan,
                   "prec_micro": np.nan, "rec_micro": np.nan, "f1_micro": np.nan},
    }
    if not both:
        return out

    # 固定阈值
    y_pred_fixed = (y_score >= fixed_thr).astype(int)
    out["fixed"]["prec_macro"] = _val(precision_score(y_true, y_pred_fixed, average='macro', zero_division=0))
    out["fixed"]["rec_macro"]  = _val(recall_score   (y_true, y_pred_fixed, average='macro', zero_division=0))
    out["fixed"]["f1_macro"]   = _val(f1_score       (y_true, y_pred_fixed, average='macro', zero_division=0))
    out["fixed"]["prec_micro"] = _val(precision_score(y_true, y_pred_fixed, average='micro', zero_division=0))
    out["fixed"]["rec_micro"]  = _val(recall_score   (y_true, y_pred_fixed, average='micro', zero_division=0))
    out["fixed"]["f1_micro"]   = _val(f1_score       (y_true, y_pred_fixed, average='micro', zero_division=0))

    # 最优F1阈值（PR曲线搜索）
    prec, rec, thr = precision_recall_curve(y_true, y_score)
    # f1 与 thr 对齐：去掉第一个点（通常 rec=0, 无阈值）
    f1 = 2 * prec * rec / np.clip(prec + rec, 1e-12, None)
    f1 = f1[1:]            # 与 thr 对齐
    if len(thr) > 0 and len(f1) == len(thr):
        best = np.nanmax(f1)
        cand = np.where(f1 == best)[0]
        best_thr = float(np.median(thr[cand]))
        out["bestF1_thr"] = best_thr

        y_pred_best = (y_score >= best_thr).astype(int)
        out["bestF1"]["prec_macro"] = _val(precision_score(y_true, y_pred_best, average='macro', zero_division=0))
        out["bestF1"]["rec_macro"]  = _val(recall_score   (y_true, y_pred_best, average='macro', zero_division=0))
        out["bestF1"]["f1_macro"]   = _val(f1_score       (y_true, y_pred_best, average='macro', zero_division=0))
        out["bestF1"]["prec_micro"] = _val(precision_score(y_true, y_pred_best, average='micro', zero_division=0))
        out["bestF1"]["rec_micro"]  = _val(recall_score   (y_true, y_pred_best, average='micro', zero_division=0))
        out["bestF1"]["f1_micro"]   = _val(f1_score       (y_true, y_pred_best, average='micro', zero_division=0))
    return out

def _fmt_or_na(v):
    return "N/A" if (v is None or (isinstance(v, float) and (np.isnan(v) or np.isinf(v)))) else f"{v:.4f}"

# ---- 推理（AMP + inference_mode + 预分配）----
N = len(valid_dataset)
if N == 0:
    print("[Step3B] ⚠️ 验证集为空，跳过。")
else:
    y_true = np.empty(N, dtype=np.int64)
    y_score = np.empty(N, dtype=np.float32)
    # 分组分析所需索引
    all_user_idx = np.empty(N, dtype=np.int64)
    all_asset_idx = np.empty(N, dtype=np.int64)
    offset = 0

    t0 = time.time()
    with torch.inference_mode():
        for batch in valid_loader_eval:
            # 假设数据集返回：user_idx, user_cat, asset_idx, asset_cat, asset_num, label
            user_idx, user_cat, asset_idx, asset_cat, asset_num, label = batch

            bs = label.size(0)
            # 保存索引到CPU数组
            if user_idx.device.type != 'cpu':
                all_user_idx[offset:offset+bs] = user_idx.cpu().numpy()
            else:
                all_user_idx[offset:offset+bs] = user_idx.numpy()

            if asset_idx.device.type != 'cpu':
                all_asset_idx[offset:offset+bs] = asset_idx.cpu().numpy()
            else:
                all_asset_idx[offset:offset+bs] = asset_idx.numpy()

            # 搬到设备
            nb = (DEVICE.type == "cuda")
            user_idx  = user_idx.to(DEVICE, non_blocking=nb)
            user_cat  = user_cat.to(DEVICE, non_blocking=nb)
            asset_idx = asset_idx.to(DEVICE, non_blocking=nb)
            asset_cat = asset_cat.to(DEVICE, non_blocking=nb)
            asset_num = asset_num.to(DEVICE, non_blocking=nb)

            # 前向
            try:
                # torch 2.x
                with torch.amp.autocast(device_type=DEVICE.type, enabled=USE_AMP):
                    logits = model(user_idx, user_cat, asset_idx, asset_cat, asset_num)
                    probs  = torch.sigmoid(logits).float()
            except Exception:
                # 兼容 torch 1.x
                with torch.cuda.amp.autocast(enabled=(USE_AMP and DEVICE.type == "cuda")):
                    logits = model(user_idx, user_cat, asset_idx, asset_cat, asset_num)
                    probs  = torch.sigmoid(logits).float()

            # 收集到CPU
            l_cpu = label.detach().cpu().numpy()
            y_true[offset:offset+bs]  = l_cpu.astype(np.int64)
            y_score[offset:offset+bs] = probs.detach().cpu().numpy().astype(np.float32)
            offset += bs

    dt = time.time() - t0
    print(f"[Step3B] 推理完成：{N} 样本，用时 {dt:.2f}s；batch={EVAL_BATCH_SIZE}, workers={NUM_WORKERS}")

    # ---- 指标（全量）----
    y_score = np.clip(y_score, 1e-7, 1 - 1e-7)
    metrics = quick_metrics(y_true, y_score)
    logloss_str = "N/A" if (metrics["logloss"] is None or np.isnan(metrics["logloss"])) else f"{metrics['logloss']:.5f}"
    print(
        "[Step3B] 指标："
        f"AUC={fmt(metrics['auc'])} | AP={fmt(metrics['ap'])} | "
        f"Logloss={logloss_str} | KS={fmt(metrics['ks'])} | Brier={metrics['brier']:.5f}"
    )

    # ---- 基于阈值的分类指标（Precision/Recall/F1：macro & micro）----
    prf = prf_with_thresholds(y_true, y_score, fixed_thr=0.5)

    print(
        "[Step3B] 分类@thr=0.5： "
        f"P_macro={_fmt_or_na(prf['fixed']['prec_macro'])} | "
        f"R_macro={_fmt_or_na(prf['fixed']['rec_macro'])} | "
        f"F1_macro={_fmt_or_na(prf['fixed']['f1_macro'])} || "
        f"P_micro={_fmt_or_na(prf['fixed']['prec_micro'])} | "
        f"R_micro={_fmt_or_na(prf['fixed']['rec_micro'])} | "
        f"F1_micro={_fmt_or_na(prf['fixed']['f1_micro'])}"
    )

    print(
        "[Step3B] 分类@thr*=BestF1： "
        f"thr*={_fmt_or_na(prf['bestF1_thr'])} | "
        f"P_macro={_fmt_or_na(prf['bestF1']['prec_macro'])} | "
        f"R_macro={_fmt_or_na(prf['bestF1']['rec_macro'])} | "
        f"F1_macro={_fmt_or_na(prf['bestF1']['f1_macro'])} || "
        f"P_micro={_fmt_or_na(prf['bestF1']['prec_micro'])} | "
        f"R_micro={_fmt_or_na(prf['bestF1']['rec_micro'])} | "
        f"F1_micro={_fmt_or_na(prf['bestF1']['f1_micro'])}"
    )

    # 保存分类报告CSV
    try:
        prf_row = {
            "thr_fixed": prf["fixed_thr"],
            "P_macro_fixed": prf["fixed"]["prec_macro"],
            "R_macro_fixed": prf["fixed"]["rec_macro"],
            "F1_macro_fixed": prf["fixed"]["f1_macro"],
            "P_micro_fixed": prf["fixed"]["prec_micro"],
            "R_micro_fixed": prf["fixed"]["rec_micro"],
            "F1_micro_fixed": prf["fixed"]["f1_micro"],
            "thr_bestF1": prf["bestF1_thr"],
            "P_macro_best": prf["bestF1"]["prec_macro"],
            "R_macro_best": prf["bestF1"]["rec_macro"],
            "F1_macro_best": prf["bestF1"]["f1_macro"],
            "P_micro_best": prf["bestF1"]["prec_micro"],
            "R_micro_best": prf["bestF1"]["rec_micro"],
            "F1_micro_best": prf["bestF1"]["f1_micro"],
            "AUC": metrics["auc"], "AP": metrics["ap"], "KS": metrics["ks"], "Brier": metrics["brier"],
            "Logloss": metrics["logloss"],
        }
        pd.DataFrame([prf_row]).to_csv("xdeepfm_cls_report.csv", index=False)
        print("[Step3B] ✅ 已保存：xdeepfm_cls_report.csv")
    except Exception as e:
        print(f"[Step3B] ⚠️ 保存分类报告失败：{e}")

    # ================================
    # 分组评估（诊断AUC虚高来源）
    # ================================
    print("\n[研究版] 分组评估分析：")

    def print_group(name, mask):
        n = int(mask.sum())
        if n <= 0:
            print(f"  {name}：无样本")
            return
        m = quick_metrics(y_true[mask], y_score[mask])
        print(
            f"  {name} (n={n}): "
            f"AUC={fmt(m['auc'])} | AP={fmt(m['ap'])} | Brier={m['brier']:.5f}"
        )
        # 分组F1
        prf_g = prf_with_thresholds(y_true[mask], y_score[mask], fixed_thr=0.5)
        print(
            "    分类@0.5: "
            f"F1_macro={_fmt_or_na(prf_g['fixed']['f1_macro'])} | "
            f"F1_micro={_fmt_or_na(prf_g['fixed']['f1_micro'])} ; "
            "分类@BestF1: "
            f"thr*={_fmt_or_na(prf_g['bestF1_thr'])} | "
            f"F1_macro={_fmt_or_na(prf_g['bestF1']['f1_macro'])} | "
            f"F1_micro={_fmt_or_na(prf_g['bestF1']['f1_micro'])}"
        )

    # 1) Warm/Cold 用户
    if 'train_df' in globals():
        train_user_set = set(pd.Series(train_df['user_idx']).unique())
        warm_user_mask = np.array([uid in train_user_set for uid in all_user_idx], dtype=bool)
        cold_user_mask = ~warm_user_mask
        print_group("Warm用户", warm_user_mask)
        print_group("Cold用户", cold_user_mask)
    else:
        print("  ⚠️ 缺少 train_df，跳过 Warm/Cold 用户分析。")

    # 2) Warm/Cold 资产
    if 'train_df' in globals():
        train_asset_set = set(pd.Series(train_df['asset_idx']).unique())
        warm_asset_mask = np.array([aid in train_asset_set for aid in all_asset_idx], dtype=bool)
        cold_asset_mask = ~warm_asset_mask
        print_group("Warm资产", warm_asset_mask)
        print_group("Cold资产", cold_asset_mask)
    else:
        print("  ⚠️ 缺少 train_df，跳过 Warm/Cold 资产分析。")

    # 3) 热门资产（按训练集出现次数 Top10%）
    if 'train_df' in globals():
        asset_popularity = pd.Series(train_df['asset_idx']).value_counts().to_dict()
        asset_pop_values = np.array([asset_popularity.get(aid, 0) for aid in all_asset_idx])

        if asset_pop_values.max() > 0:
            pos_mask = (asset_pop_values > 0)
            if pos_mask.any():
                pop_q90 = np.percentile(asset_pop_values[pos_mask], 90)
                hot_asset_mask = (asset_pop_values >= pop_q90)
                normal_asset_mask = (asset_pop_values > 0) & (asset_pop_values < pop_q90)
                print_group("热门资产Top10%", hot_asset_mask)
                print_group("普通资产", normal_asset_mask)
            else:
                print("  热门资产分析：验证集中资产从未在训练出现，跳过。")
        else:
            print("  热门资产分析：验证集中资产热度为 0，跳过。")
    else:
        print("  ⚠️ 缺少 train_df，跳过热门度分组。")

    # 4) 共现分析：训练集见过的 (user, asset) 对
    if 'train_df' in globals():
        train_pairs = set(zip(train_df['user_idx'], train_df['asset_idx']))
        seen_pair_mask = np.array([(uid, aid) in train_pairs for uid, aid in zip(all_user_idx, all_asset_idx)], dtype=bool)
        unseen_pair_mask = ~seen_pair_mask
        print_group("训练集见过的(user,asset)对", seen_pair_mask)
        print_group("训练集未见的(user,asset)对", unseen_pair_mask)
    else:
        print("  ⚠️ 缺少 train_df，跳过共现分析。")

    # ---- 绘图并保存（ROC / PR），仅在两类都存在时绘制 ----
    pos_all, neg_all = _class_counts(y_true)
    both_all = (pos_all > 0 and neg_all > 0)

    if both_all:
        fpr, tpr, _ = roc_curve(y_true, y_score)
        roc_auc = auc(fpr, tpr)
        plt.figure()
        plt.plot(fpr, tpr, lw=2, label=f"ROC (AUC = {roc_auc:.4f})")
        plt.plot([0,1],[0,1],'--',lw=1.2)
        plt.xlim([0,1]); plt.ylim([0,1.05])
        plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate")
        plt.title("Receiver Operating Characteristic")
        plt.legend(loc="lower right"); plt.grid(alpha=0.3)
        plt.tight_layout(); plt.savefig("xdeepfm_ROC.png", dpi=150); plt.close()
        print("\n[Step3B] ✅ 已保存：xdeepfm_ROC.png")

        precision, recall, _ = precision_recall_curve(y_true, y_score)
        ap = average_precision_score(y_true, y_score)
        plt.figure()
        plt.plot(recall, precision, lw=2, label=f"PR (AP = {ap:.4f})")
        plt.xlim([0,1]); plt.ylim([0,1.05])
        plt.xlabel("Recall"); plt.ylabel("Precision")
        plt.title("Precision–Recall Curve")
        plt.legend(loc="lower left"); plt.grid(alpha=0.3)
        plt.tight_layout(); plt.savefig("xdeepfm_PR.png", dpi=150); plt.close()
        print("[Step3B] ✅ 已保存：xdeepfm_PR.png")
    else:
        print("\n[Step3B] ⚠️ 验证集只有单一类别，跳过 ROC/PR 曲线。")

    # ---- 保存中间结果，后续画图无需再推理 ----
    np.save("valid_y_true.npy",  y_true)
    np.save("valid_y_score.npy", y_score)
    print("[Step3B] 缓存：valid_y_true.npy / valid_y_score.npy")


[Step3B] 开始评估与绘图（使用验证集）
[Step3B] 已加载最佳权重：xdeepfm_best.pt
[Step3B] 推理完成：89884 样本，用时 12.74s；batch=2048, workers=8
[Step3B] 指标：AUC=0.8988 | AP=0.7229 | Logloss=0.50573 | KS=0.0000 | Brier=0.16279
[Step3B] 分类@thr=0.5： P_macro=0.7180 | R_macro=0.8155 | F1_macro=0.7325 || P_micro=0.7833 | R_micro=0.7833 | F1_micro=0.7833
[Step3B] 分类@thr*=BestF1： thr*=0.6758 | P_macro=0.7705 | R_macro=0.8067 | F1_macro=0.7858 || P_micro=0.8534 | R_micro=0.8534 | F1_micro=0.8534
[Step3B] ✅ 已保存：xdeepfm_cls_report.csv

[研究版] 分组评估分析：
  Warm用户 (n=89854): AUC=0.8988 | AP=0.7230 | Brier=0.16276
    分类@0.5: F1_macro=0.7326 | F1_micro=0.7833 ; 分类@BestF1: thr*=0.6758 | F1_macro=0.7858 | F1_micro=0.8534
  Cold用户 (n=30): AUC=0.7500 | AP=0.4710 | Brier=0.25496
    分类@0.5: F1_macro=0.5971 | F1_micro=0.6333 ; 分类@BestF1: thr*=0.6445 | F1_macro=0.6591 | F1_micro=0.7333
  Warm资产 (n=89864): AUC=0.8988 | AP=0.7230 | Brier=0.16277
    分类@0.5: F1_macro=0.7326 | F1_micro=0.7833 ; 分类@BestF1: thr*=0.6758 | F1_macro=0.7858 | F1_micro

In [12]:
# ================================
# Step 3C：按用户的排序指标（Hit@K / MRR / NDCG@K）
# 依赖：model, DEVICE, USE_AMP, valid_dataset
# ================================
import numpy as np
import torch
from torch.utils.data import DataLoader
from collections import defaultdict
from sklearn.metrics import roc_auc_score

K_LIST = [5, 10]  # 可改为 [5, 10, 20]

def make_eval_loader_for_userwise(dataset):
    num_workers = 4 if USE_AMP else 0
    return DataLoader(
        dataset,
        batch_size=(2048 if USE_AMP else 512),
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True if USE_AMP else False,
        persistent_workers=True if (USE_AMP and num_workers>0) else False,
        prefetch_factor=(2 if (USE_AMP and num_workers>0) else None),
    )

@torch.no_grad()
def collect_valid_arrays():
    loader = make_eval_loader_for_userwise(valid_dataset)
    model.eval().to(DEVICE)
    users, items, labels, scores = [], [], [], []
    with torch.inference_mode():
        for batch in loader:
            u, ucat, a, acat, anum, y = batch
            u_d = u.to(DEVICE, non_blocking=USE_AMP)
            ucat_d = ucat.to(DEVICE, non_blocking=USE_AMP)
            a_d = a.to(DEVICE, non_blocking=USE_AMP)
            acat_d = acat.to(DEVICE, non_blocking=USE_AMP)
            anum_d = anum.to(DEVICE, non_blocking=USE_AMP)
            with torch.amp.autocast(device_type=DEVICE.type, enabled=USE_AMP):
                logit = model(u_d, ucat_d, a_d, acat_d, anum_d)
                prob = torch.sigmoid(logit).float().detach().cpu().numpy()
            users.append(u.cpu().numpy()); items.append(a.cpu().numpy())
            labels.append(y.cpu().numpy()); scores.append(prob)
    users = np.concatenate(users)
    items = np.concatenate(items)
    labels = np.concatenate(labels).astype(np.int64)
    scores = np.concatenate(scores).astype(np.float32)
    return users, items, labels, scores

def dcg_at_k(rel_sorted, k):
    """支持样本数少于 k 的情况"""
    if rel_sorted.size == 0:
        return 0.0
    eff_k = min(k, rel_sorted.size)
    rel = rel_sorted[:eff_k]
    denom = np.log2(np.arange(2, eff_k + 2))
    return float(np.sum((2.0**rel - 1.0) / denom))

def ndcg_at_k(rel_sorted, k):
    """支持样本数少于 k 的情况"""
    if rel_sorted.size == 0:
        return 0.0
    eff_k = min(k, rel_sorted.size)
    ideal = np.sort(rel_sorted)[::-1][:eff_k]
    # 这里把 k 换成 eff_k 传给 dcg，确保分子分母同长度
    return dcg_at_k(rel_sorted, eff_k) / max(dcg_at_k(ideal, eff_k), 1e-9)

def compute_userwise_metrics(users, items, labels, scores, K_list=(5,10)):
    # 按用户聚合候选（仅验证集中已有的样本）
    by_user = defaultdict(list)
    for u, it, y, s in zip(users, items, labels, scores):
        by_user[int(u)].append((int(it), int(y), float(s)))

    hits = {k: [] for k in K_list}
    mrrs = []
    ndcgs = {k: [] for k in K_list}

    valid_users = 0
    for u, lst in by_user.items():
        # 该用户在验证集里如果没有任何正例，就跳过
        if not any(y==1 for _, y, _ in lst):
            continue
        valid_users += 1

        # 按概率从高到低排序
        lst_sorted = sorted(lst, key=lambda x: x[2], reverse=True)
        rel = np.array([y for _, y, _ in lst_sorted], dtype=np.float32)

        # Hit@K（考虑样本数少于 k）
        for k in K_list:
            eff_k = min(k, rel.size)
            hit = float(rel[:eff_k].max()) if eff_k > 0 else 0.0
            hits[k].append(hit)

        # MRR：第一个正例的倒数排名
        rr = 0.0
        for rank, y in enumerate(rel, start=1):
            if y > 0.5:
                rr = 1.0 / rank
                break
        mrrs.append(rr)

        # NDCG@K（内部已处理 eff_k）
        for k in K_list:
            ndcgs[k].append(ndcg_at_k(rel, k))

    out = {
        "num_users_with_positive": valid_users,
        "Hit@K": {k: float(np.mean(hits[k])) if hits[k] else float("nan") for k in K_list},
        "MRR": float(np.mean(mrrs)) if mrrs else float("nan"),
        "NDCG@K": {k: float(np.mean(ndcgs[k])) if ndcgs[k] else float("nan") for k in K_list},
    }
    return out

# —— 执行 —— #
print("\n[Step3C] 开始按用户的排序指标评估（基于验证集中已有候选）...")
u_all, i_all, y_all, s_all = collect_valid_arrays()
auc_overall = roc_auc_score(y_all, s_all) if (y_all.sum()>0 and (1-y_all).sum()>0) else float("nan")
userwise = compute_userwise_metrics(u_all, i_all, y_all, s_all, K_list=K_LIST)

print(f"[Step3C] Overall AUC = {auc_overall:.4f}")
print(f"[Step3C] Users with >=1 positive in valid = {userwise['num_users_with_positive']}")
for k, v in userwise["Hit@K"].items():
    print(f"[Step3C] Hit@{k} = {v:.4f}")
print(f"[Step3C] MRR = {userwise['MRR']:.4f}")
for k, v in userwise["NDCG@K"].items():
    print(f"[Step3C] NDCG@{k} = {v:.4f}")
print("[Step3C] 完成（仅作研究观察，不影响训练/导出）。")


[Step3C] 开始按用户的排序指标评估（基于验证集中已有候选）...
[Step3C] Overall AUC = 0.8988
[Step3C] Users with >=1 positive in valid = 10968
[Step3C] Hit@5 = 0.9957
[Step3C] Hit@10 = 0.9995
[Step3C] MRR = 0.9217
[Step3C] NDCG@5 = 0.9195
[Step3C] NDCG@10 = 0.9286
[Step3C] 完成（仅作研究观察，不影响训练/导出）。


In [13]:
# ================================
# Step 4A: 训练曲线可视化（Loss / AUC / Logloss）【研究版】
# 依赖: history = {"train_loss": [], "valid_loss": [], "valid_auc": [], "valid_logloss": []}
# 学习率曲线、过拟合程度分析
# ================================
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from typing import List, Tuple, Dict

def _clean_series(y: List[float]) -> np.ndarray:
    """把列表转成 ndarray，并去掉头尾的 NaN（中间 NaN 保留，避免错位）"""
    if y is None:
        return np.array([])
    arr = np.asarray(y, dtype=float)
    if arr.size == 0:
        return arr
    if np.isnan(arr).all():
        return np.array([])
    # 去掉前后 NaN，保留中间 NaN
    first = np.argmax(~np.isnan(arr))
    last = len(arr) - np.argmax(~np.isnan(arr[::-1]))
    return arr[first:last]

def _plot_xy(x: np.ndarray, ys: List[Tuple[np.ndarray, str]], title: str, ylabel: str,
             save_path: Path, ylim: Tuple[float, float] = None):
    """通用画图：支持单点，用 marker，自动网格与紧凑布局"""
    plt.figure(figsize=(8, 6))
    drew = False
    for y, label in ys:
        if y.size == 0:
            continue
        n = min(x.size, y.size)
        if n == 0:
            continue
        xx, yy = x[:n], y[:n]
        plt.plot(xx, yy, label=label, marker='o', linewidth=1.5, markersize=4)
        drew = True

    if not drew:
        plt.close()
        return False

    plt.xlabel("Epoch")
    plt.ylabel(ylabel)
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.grid(alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.savefig(str(save_path), dpi=150)
    plt.close()
    return True

def plot_training_curves(history: dict, save_prefix="xdeepfm_training", save_dir: str = "."):
    save_dir = Path(save_dir)
    save_dir.mkdir(parents=True, exist_ok=True)

    # 取出并清洗
    tl  = _clean_series(history.get("train_loss"))
    vl  = _clean_series(history.get("valid_loss"))
    va  = _clean_series(history.get("valid_auc"))
    vll = _clean_series(history.get("valid_logloss"))

    # 统一 x 轴（以最长的一个为准）
    max_len = max([arr.size for arr in [tl, vl, va, vll]] + [0])
    if max_len == 0:
        print("⚠️ history 为空或全 NaN，未生成任何曲线。")
        return

    epochs = np.arange(1, max_len + 1)
    saved = []

    # 1) Loss曲线
    out_loss = save_dir / f"{save_prefix}_loss.png"
    if _plot_xy(epochs,
                [(tl, "Train Loss"), (vl, "Valid Loss")],
                title="Training vs Validation Loss",
                ylabel="Loss",
                save_path=out_loss):
        saved.append(out_loss.name)

    # 2) AUC曲线
    out_auc = save_dir / f"{save_prefix}_auc.png"
    if _plot_xy(epochs,
                [(va, "Valid AUC")],
                title="Validation AUC",
                ylabel="AUC",
                save_path=out_auc,
                ylim=(0.0, 1.0)):
        saved.append(out_auc.name)

    # 3) Logloss曲线
    out_logloss = save_dir / f"{save_prefix}_logloss.png"
    if vll.size > 0 and _plot_xy(epochs,
                                 [(vll, "Valid Logloss")],
                                 title="Validation Logloss",
                                 ylabel="Logloss",
                                 save_path=out_logloss):
        saved.append(out_logloss.name)

    # ================================
    # 诊断性可视化
    # ================================
    
    # 4) 过拟合程度分析（Train vs Valid Loss差异）
    if tl.size > 0 and vl.size > 0:
        n = min(tl.size, vl.size)
        overfitting_gap = vl[:n] - tl[:n]
        
        out_overfit = save_dir / f"{save_prefix}_overfitting.png"
        plt.figure(figsize=(8, 6))
        plt.plot(epochs[:n], overfitting_gap, 'r-o', label='Valid Loss - Train Loss', 
                linewidth=1.5, markersize=4)
        plt.axhline(y=0, color='k', linestyle='--', alpha=0.5)
        plt.xlabel("Epoch")
        plt.ylabel("Loss Gap")
        plt.title("Overfitting Analysis (Gap = Valid - Train)")
        plt.grid(alpha=0.3)
        plt.legend()
        plt.tight_layout()
        plt.savefig(str(out_overfit), dpi=150)
        plt.close()
        saved.append(out_overfit.name)
        
        # 打印过拟合诊断
        final_gap = overfitting_gap[-1] if n > 0 else 0
        print(f"\n[研究版] 过拟合分析：")
        print(f"  最终Loss差距: {final_gap:.5f}")
        if final_gap < 0.01:
            print("  诊断: 基本无过拟合")
        elif final_gap < 0.05:
            print("  诊断: 轻度过拟合")
        elif final_gap < 0.1:
            print("  诊断: 中度过拟合")
        else:
            print("  诊断: 严重过拟合，建议增加正则化")
    
    # 5) 组合诊断图（2x2子图）
    if any([tl.size > 0, vl.size > 0, va.size > 0, vll.size > 0]):
        out_combined = save_dir / f"{save_prefix}_combined.png"
        fig, axes = plt.subplots(2, 2, figsize=(12, 10))
        fig.suptitle('Training Progress Overview', fontsize=16)
        
        # 子图1: Loss
        ax1 = axes[0, 0]
        if tl.size > 0:
            ax1.plot(epochs[:tl.size], tl, 'b-o', label='Train Loss', markersize=4)
        if vl.size > 0:
            ax1.plot(epochs[:vl.size], vl, 'r-o', label='Valid Loss', markersize=4)
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Loss')
        ax1.set_title('Loss Curves')
        ax1.grid(alpha=0.3)
        ax1.legend()
        
        # 子图2: AUC
        ax2 = axes[0, 1]
        if va.size > 0:
            ax2.plot(epochs[:va.size], va, 'g-o', label='Valid AUC', markersize=4)
            ax2.set_ylim(0.0, 1.0)
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('AUC')
        ax2.set_title('Validation AUC')
        ax2.grid(alpha=0.3)
        ax2.legend()
        
        # 子图3: LogLoss
        ax3 = axes[1, 0]
        if vll.size > 0:
            ax3.plot(epochs[:vll.size], vll, 'm-o', label='Valid LogLoss', markersize=4)
        ax3.set_xlabel('Epoch')
        ax3.set_ylabel('LogLoss')
        ax3.set_title('Validation LogLoss')
        ax3.grid(alpha=0.3)
        ax3.legend()
        
        # 子图4: 早停指标（假设用AUC）
        ax4 = axes[1, 1]
        if va.size > 0:
            # 标记最佳epoch
            best_epoch = np.argmax(va) + 1
            ax4.plot(epochs[:va.size], va, 'b-o', markersize=4)
            ax4.plot(best_epoch, va[best_epoch-1], 'r*', markersize=15, 
                    label=f'Best @ Epoch {best_epoch}')
            ax4.set_ylim(0.0, 1.0)
        ax4.set_xlabel('Epoch')
        ax4.set_ylabel('Metric')
        ax4.set_title('Early Stopping Monitor (AUC)')
        ax4.grid(alpha=0.3)
        ax4.legend()
        
        plt.tight_layout()
        plt.savefig(str(out_combined), dpi=150)
        plt.close()
        saved.append(out_combined.name)
    
    # 打印训练摘要
    print(f"\n[研究版] 训练摘要：")
    if va.size > 0:
        best_auc_epoch = np.argmax(va) + 1
        print(f"  最佳AUC: {va[best_auc_epoch-1]:.5f} @ Epoch {best_auc_epoch}")
        print(f"  最终AUC: {va[-1]:.5f}")
        if va[-1] < va[best_auc_epoch-1] - 0.001:
            print("  提示: 最终模型不是最优，可能过拟合")
    
    if vll.size > 0:
        best_ll_epoch = np.argmin(vll) + 1
        print(f"  最佳LogLoss: {vll[best_ll_epoch-1]:.5f} @ Epoch {best_ll_epoch}")
        print(f"  最终LogLoss: {vll[-1]:.5f}")

    if saved:
        print(f"\n训练曲线已保存： {' / '.join(saved)}")
    else:
        print("⚠️ 指标不可用，未生成任何曲线。")

# 调用
plot_training_curves(history, save_prefix="xdeepfm_training", save_dir=".")


[研究版] 过拟合分析：
  最终Loss差距: 0.19487
  诊断: 严重过拟合，建议增加正则化

[研究版] 训练摘要：
  最佳AUC: 0.89877 @ Epoch 2
  最终AUC: 0.87766
  提示: 最终模型不是最优，可能过拟合
  最佳LogLoss: 0.46923 @ Epoch 4
  最终LogLoss: 0.48301

训练曲线已保存： xdeepfm_training_loss.png / xdeepfm_training_auc.png / xdeepfm_training_logloss.png / xdeepfm_training_overfitting.png / xdeepfm_training_combined.png


In [14]:
# ================================
# Step 4B 研究版：全量偏好矩阵导出（无全量微调）
# - 直接加载 BEST_PATH 权重
# - 导出为“宽矩阵”：第一列 customerID，第一行 ISIN
# - 保持 GPU 自动加速 / CPU 兼容
# 依赖：
# - model, BEST_PATH, user2idx, asset2idx
# - user_cat_features, asset_cat_features, num_cols
# - customers_latest, assets_latest
# ================================

import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
USE_AMP = (DEVICE.type == "cuda")
BF16_OK = (USE_AMP and torch.cuda.is_bf16_supported())
AMP_DTYPE = torch.bfloat16 if BF16_OK else torch.float16

# 加载最佳模型权重
if os.path.exists(BEST_PATH):
    state = torch.load(BEST_PATH, map_location="cpu")
    model.load_state_dict(state if isinstance(state, dict) else state.state_dict())
    print(f"[Step4B 研究版] 加载最佳验证模型权重：{BEST_PATH}")
else:
    print(f"[Step4B 研究版] WARNING：未找到 {BEST_PATH}，使用当前模型权重")

model.to(DEVICE)
model.eval()

# 分块导出配置（按设备）
USER_BATCH_GPU = 256
ITEM_BATCH_GPU = 256
USER_BATCH_CPU = 128
ITEM_BATCH_CPU = 128

USER_BATCH = USER_BATCH_GPU if USE_AMP else USER_BATCH_CPU
ITEM_BATCH = ITEM_BATCH_GPU if USE_AMP else ITEM_BATCH_CPU

OUT_CSV = "user_asset_preferences_full.csv"

# 反向索引
idx2user = {v: k for k, v in user2idx.items()}
idx2asset = {v: k for k, v in asset2idx.items()}

# 预取特征表（确保按 idx 可 .loc）
all_users_cat = customers_latest.set_index('user_idx')[user_cat_features].sort_index()
all_assets_cat_num = assets_latest.set_index('asset_idx')[asset_cat_features + num_cols].sort_index()

all_user_indices = np.arange(len(user2idx))
all_asset_indices = np.arange(len(asset2idx))

# ================================
# 将“长表”导出改为“宽矩阵”导出（第一列用户，第一行资产）
# ================================

# 列轴：按 asset_idx 顺序取出所有 ISIN
all_isins = [idx2asset[int(i)] for i in all_asset_indices]

# 删除旧文件，并写入表头（customerID + 所有 ISIN）
if os.path.exists(OUT_CSV):
    os.remove(OUT_CSV)

header = ["customerID"] + all_isins
pd.DataFrame([header]).to_csv(OUT_CSV, index=False, header=False)
print(f"[Step4B 研究版] 矩阵表头写入：{len(header)} 列（含 customerID）")

total_users_written = 0
print("[Step4B 研究版] 开始按用户批次生成【宽矩阵】概率分数…")

# 用 inference_mode 比 no_grad 更省同步
with torch.inference_mode():
    for ui in tqdm(range(0, len(all_user_indices), USER_BATCH), desc="用户批次"):
        # 取一批用户
        u_idx_batch = all_user_indices[ui: ui + USER_BATCH].tolist()
        U = len(u_idx_batch)

        # (U,) 用户索引与特征（放在 DEVICE 上）
        user_batch_idx = torch.tensor(u_idx_batch, dtype=torch.long, device=DEVICE)
        user_cat_batch = torch.tensor(
            all_users_cat.loc[u_idx_batch].values, dtype=torch.long, device=DEVICE
        )  # (U, num_user_cat)

        # 为了省显存：全资产按 ITEM_BATCH 分块推理，横向拼接
        probs_blocks = []
        for ai in range(0, len(all_asset_indices), ITEM_BATCH):
            a_idx_batch = all_asset_indices[ai: ai + ITEM_BATCH].tolist()
            A = len(a_idx_batch)

            asset_batch_idx = torch.tensor(a_idx_batch, dtype=torch.long, device=DEVICE)
            asset_cat_batch = torch.tensor(
                all_assets_cat_num.loc[a_idx_batch][asset_cat_features].values,
                dtype=torch.long, device=DEVICE
            )  # (A, num_asset_cat)
            asset_num_batch = torch.tensor(
                all_assets_cat_num.loc[a_idx_batch][num_cols].values,
                dtype=torch.float32, device=DEVICE
            )  # (A, num_num_cols)

            # 组装成 U×A 对齐的扁平输入，然后再 reshape 回 (U, A)
            user_expand_idx  = user_batch_idx.unsqueeze(1).repeat(1, A).reshape(-1)               # (U*A,)
            asset_expand_idx = asset_batch_idx.unsqueeze(0).repeat(U, 1).reshape(-1)              # (U*A,)
            user_cat_expand  = user_cat_batch.unsqueeze(1).repeat(1, A, 1).reshape(-1, user_cat_batch.size(1))  # (U*A, _)
            asset_cat_expand = asset_cat_batch.unsqueeze(0).repeat(U, 1, 1).reshape(-1, asset_cat_batch.size(1)) # (U*A, _)
            asset_num_expand = asset_num_batch.unsqueeze(0).repeat(U, 1, 1).reshape(-1, asset_num_batch.size(1)) # (U*A, _)

            # AMP 下可能是 BF16，转成 FP32 再落盘
            with torch.amp.autocast(device_type=DEVICE.type, enabled=USE_AMP, dtype=AMP_DTYPE):
                logits = model(user_expand_idx, user_cat_expand, asset_expand_idx, asset_cat_expand, asset_num_expand)
                probs  = torch.sigmoid(logits).to(torch.float32)

            probs_chunk = probs.detach().cpu().numpy().reshape(U, A)  # 还原成 (U, A_chunk)
            probs_blocks.append(probs_chunk)

        # 横向拼接所有资产块 -> (U, |all_assets|)
        probs_full = np.concatenate(probs_blocks, axis=1).astype(np.float32)

        # 生成本批 DataFrame（按“宽矩阵”结构），第一列 customerID
        df_chunk = pd.DataFrame(probs_full, columns=all_isins)
        df_chunk.insert(0, "customerID", [idx2user[int(i)] for i in u_idx_batch])

        # 以追加方式写入（表头已写过，这里不再写 header）
        df_chunk.to_csv(OUT_CSV, mode="a", index=False, header=False, float_format="%.6f")

        total_users_written += U

print(f"[Step4B 研究版] 宽矩阵已导出到：{OUT_CSV}")
print(f"共写入用户 {total_users_written:,}，矩阵维度约为 ({total_users_written:,}, {len(all_isins)+1:,})（含 customerID 列）")

[Step4B 研究版] 加载最佳验证模型权重：xdeepfm_best.pt
[Step4B 研究版] 矩阵表头写入：807 列（含 customerID）
[Step4B 研究版] 开始按用户批次生成【宽矩阵】概率分数…


用户批次: 100%|██████████| 114/114 [00:32<00:00,  3.54it/s]

[Step4B 研究版] 宽矩阵已导出到：user_asset_preferences_full.csv
共写入用户 29,090，矩阵维度约为 (29,090, 807)（含 customerID 列）
